## Network validation
This scripts is meant to validate that the designed vertex location match the real locations.

The theoratical vertices are found in a network model as net.vertices.
The real vertices are retrieved from a picture of the network suspended in a frame. The following steps are required to enable the comparison:
- Conventional cameras have some distortion. (straight lines appear to be curved). The camera distortion can be found from a set of images of a checkerboard with known dimensions. The image is then undistorted.
- Ideally, a picture of the network is taken exaclty orthogonal to the subject. The homography of the camera angle with respect to the frame is determined. The frame contains markers on a straight plane with known world coordinates. With the homography known an image can be warped such that it appears to take the frame exactly orthogonal to the frame.

Finally, the vertices are selected in the image and they can be compared to their theoratical locations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import glob
from src.network import Network_custom

### Open the network model

In [27]:
BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"
# model_name  = 'Validation_structure_1'
model_name = 'VS_ring0.5_connectors0.8_center0.2_as1.5_asr1.5_s110'
net = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))
net.net_plot(color=True, elables = True, vlabels = False)

picture_number = 0
model_name_im = model_name + f"_{picture_number}"


### Load a set of images of checker boards
Internal camera and distortion parameters are determined of the camera. The process needs to be repeated when changing anything on the camera set up (lens/zoom/aperture/etc..). Make sure that auto-focus is off

In [3]:
# checkerboard_size = (8, 12)  # Number of inner corners per a chessboard row and column
# square_size = 10 #6.85/(checkerboard_size[0]+1) * 0.0254

# # Termination criteria for corner sub-pixel accuracy
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# # Prepare object points based on the real-world dimensions of the checkerboard
# objp = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
# objp[:, :2] = np.mgrid[0:checkerboard_size[1], 0:checkerboard_size[0]].T.reshape(-1, 2)
# objp *= square_size

# # Arrays to store object points and image points from all the images
# objpoints = []  # 3d points in real-world space
# imgpoints = []  # 2d points in image plane

# # Load images
# # images = glob.glob(os.path.join(BYU_UW_root,'Calibration images/*.jpg' ))
# images = glob.glob(os.path.join(BYU_UW_root,'Calibration images/new/*.jpg' ))

# for fname in images:
#     img = cv2.imread(fname)
#     img_shape = img.shape[:2]
#     img_aspect_ratio = img_shape[1] / img_shape[0]
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#     # Find the chessboard corners
#     ret, corners = cv2.findChessboardCorners( 
#                     gray, (12,8),  
#                     cv2.CALIB_CB_ADAPTIVE_THRESH  
#                     + cv2.CALIB_CB_FAST_CHECK + 
#                     cv2.CALIB_CB_NORMALIZE_IMAGE) 
#     # If found, add object points, image points (after refining them)
#     if ret:
#         objpoints.append(objp)
#         corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
#         imgpoints.append(corners2)

#         # Draw and display the corners
#         cv2.drawChessboardCorners(img, checkerboard_size, corners2, ret)
#         resized_image = cv2.resize(img, (400, int(400 / img_aspect_ratio)))
#         cv2.imshow('img', resized_image)
#         cv2.waitKey(1)
        
# cv2.destroyAllWindows()

# # Calibrate the camera
# ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
# # Print the camera matrix and distortion coefficients
# print("Camera matrix:\n", mtx)
# print("Distortion coefficients:\n", dist)

# # Check calibration results. A good mean projection error is <1.0
# mean_error = 0
# for i in range(len(objpoints)):
#     imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
#     error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
#     mean_error += error
# print("Mean reprojection error:", mean_error / len(objpoints))

# img = cv2.imread(images[-1])
# fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# ax[0].imshow(img)
# ax[0].set_title('Original Image')
# ax[1].imshow(cv2.undistort(img, mtx, dist))
# ax[1].set_title('Undistorted Image')
# plt.show()


### Load an image of the frame and click on the markers

In [28]:
import matplotlib.pyplot as plt
import os

%matplotlib qt

# Load image
picture_number = 0
image_name = model_name_im + ".jpg"
image = plt.imread(os.path.join(BYU_UW_root, 'measuerement images', image_name))
height, width, _ = image.shape
points = []

# Initial axis limits
xlim_init, ylim_init = (0, width), (height, 0)

def on_click(event):
    """Reset view when clicking anywhere in the figure."""
    if event.button == 1:  # Left mouse button
        points.append((event.xdata, event.ydata))
        ax.plot(event.xdata, event.ydata, 'r.', label='real vertices')
        ax.set_xlim(xlim_init)
        ax.set_ylim(ylim_init)
        ax.figure.canvas.draw()

def on_scroll(event):
    """Zoom in/out at mouse position using the scroll wheel."""
    scale_factor = 1.3  # Zoom speed
    xlim, ylim = ax.get_xlim(), ax.get_ylim()

    if event.step > 0:  # Zoom in
        xlim_new = (event.xdata - (event.xdata - xlim[0]) / scale_factor,
                    event.xdata + (xlim[1] - event.xdata) / scale_factor)
        ylim_new = (event.ydata - (event.ydata - ylim[0]) / scale_factor,
                    event.ydata + (ylim[1] - event.ydata) / scale_factor)
    else:  # Zoom out
        xlim_new = (event.xdata - (event.xdata - xlim[0]) * scale_factor,
                    event.xdata + (xlim[1] - event.xdata) * scale_factor)
        ylim_new = (event.ydata - (event.ydata - ylim[0]) * scale_factor,
                    event.ydata + (ylim[1] - event.ydata) * scale_factor)

    ax.set_xlim(xlim_new)
    ax.set_ylim(ylim_new)
    fig.canvas.draw()

def on_motion(event):
    """Update ring cursor position."""
    if event.xdata is not None and event.ydata is not None:
        circle_outer.set_center((event.xdata, event.ydata))
        circle_inner.set_center((event.xdata, event.ydata))
        fig.canvas.draw_idle()

def on_close(event):
    """Callback function to stop the event loop when the figure is closed."""
    plt.close()

# Display image
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image)
ax.set_xlim(xlim_init)
ax.set_ylim(ylim_init)
ax.axis("off")

# Add quadrant markers
ax.text(width/2, height*.9, '0', color='r')
ax.text(width*.9, height/2, '1', color='r')
ax.text(width/2, height*.1, '2', color='r')
ax.text(width*.1, height/2, '3', color='r')
ax.set_title("Click on markers (in order). Zoom with scroll wheel.")

# Create ring cursor
circle_outer = plt.Circle((0, 0), radius=40, color='r', fill=False, lw=2)
circle_inner = plt.Circle((0, 0), radius=20, color='r', fill=False, lw=2)
ax.add_patch(circle_outer)
ax.add_patch(circle_inner)

# Connect events
fig.canvas.mpl_connect("button_press_event", on_click)
fig.canvas.mpl_connect("scroll_event", on_scroll)
fig.canvas.mpl_connect("motion_notify_event", on_motion)

plt.show()


Use this points if you don't want to reassign the points everytime you run the script

In [29]:
# points = ([(3382.917796079749, 3466.400942109362),
#   (4818.966106172992, 2030.9365791233422),
#   (3367.876695509721, 609.515216381255),
#   (1932.9951576914789, 2043.1452614796335)])

mtx = np.array([[2.25105751e+04, 0.00000000e+00, 2.49549539e+03],
        [0.00000000e+00, 2.25183605e+04, 2.84911943e+03],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

dist = np.array([[ 0.56985739,  1.09785385, -0.00920809,  0.02059732,  3.93628828]])

points, mtx, dist

([(3383.5654904509615, 3466.371949768974),
  (4819.533731917238, 2031.0587346789432),
  (3368.06938921559, 610.6392292409334),
  (1932.9137914670014, 2043.1004948520754)],
 array([[2.25105751e+04, 0.00000000e+00, 2.49549539e+03],
        [0.00000000e+00, 2.25183605e+04, 2.84911943e+03],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[ 0.56985739,  1.09785385, -0.00920809,  0.02059732,  3.93628828]]))

### Calibrate the camera angle with the known coordinates of the markers.
This will break if the order of the coordinates are misaligned. Also notice that the y-axis is inverted when working with images, ensure your know cordinate list is set up accordingly

In [30]:
points_nonaligned    = np.array(points)
distance_between_markers_mm = 148.49 # mm 155.669 148.49mm
distance_between_markers_px = np.sqrt((points_nonaligned[3, 0] - points_nonaligned[0, 0])**2 + (points_nonaligned[3, 1] - points_nonaligned[0, 1])**2)

mm_to_px = distance_between_markers_px/distance_between_markers_mm

R_45 = np.array([[0.70710678, -0.70710678], [0.70710678, 0.70710678]])
points_aligned  = np.array([(0, -distance_between_markers_mm/2), (distance_between_markers_mm/2, 0), (0, distance_between_markers_mm/2), (-distance_between_markers_mm/2, 0)]) @ R_45 * mm_to_px + np.array([width/2, height/2]) 

print("Pixel locations:", points_nonaligned)
print("cordinate list:", points_aligned)

Pixel locations: [[3383.56549045 3466.37194977]
 [4819.53373192 2031.05873468]
 [3368.06938922  610.63922924]
 [1932.91379147 2043.10049485]]
cordinate list: [[2281.48660657 1281.48660657]
 [3718.51339343 1281.48660657]
 [3718.51339343 2718.51339343]
 [2281.48660657 2718.51339343]]


In [31]:
homography_matrix, _ = cv2.findHomography(points_nonaligned, points_aligned, method=cv2.RANSAC)
aligned_image = cv2.warpPerspective(image, homography_matrix, (width, height))

points_aligned_validate = np.array([points_nonaligned[:, 0], points_nonaligned[:, 1], np.ones_like(points_nonaligned[:, 0])])
points_aligned_validate = np.dot(homography_matrix, points_aligned_validate)
points_aligned_validate = np.array([points_aligned_validate[0, :]/points_aligned_validate[2, :], points_aligned_validate[1, :]/points_aligned_validate[2, :]]).T

fig, ax = plt.subplots(1, 2, figsize=(15, 10), sharex=True, sharey=True)
ax[0].imshow(image)
ax[0].plot(points_nonaligned[:, 0], points_nonaligned[:, 1], 'r*', markersize=10)
ax[0].set_title('Unaligned Image')
ax[0].axis('off')
ax[1].imshow(aligned_image)
ax[1].plot(points_aligned[:, 0], points_aligned[:, 1], 'r*', markersize=10, label='known points')
ax[1].plot(points_aligned_validate[:, 0], points_aligned_validate[:, 1], 'g.', markersize=10, label='known points - validate')
ax[1].set_title('Orthogonal Image')
ax[1].axis('off')
plt.legend()

In [32]:
net.vertices *= mm_to_px # Scale the network vertices to match the image scale
net.vertices += np.array([width/2, height/2,0]) # Shift the network vertices to match the image origin

In [10]:
points = []
fig, ax = plt.subplots(figsize=(15, 10))
ax.axis('off')
ax.set_aspect('equal')
ax.imshow(aligned_image)
ax = net.net_plot_mat(ax, vlabels=True, fp=True)
fig.canvas.mpl_connect('button_press_event', on_click)
fig.canvas.mpl_connect('scroll_event', on_scroll)
fig.canvas.mpl_connect('close_event', on_close)
ax.set_title('Click on the vertices of the network (in order). (Scroll to zoom)')
plt.show()

In [33]:
points = [(1580.2625829719882, 2006.2332587773935),
 (2587.1554413730464, 2129.3895074984475),
 (3303.883400693637, 1981.3638255553901),
 (3488.20262126068, 1972.3808912601758),
 (4413.030722076421, 2010.0458795768866),
 (2994.4072290497675, 588.0261617342247),
 (3040.4411359702813, 1434.1709473039678),
 (2893.520672163949, 2446.3018650937975),
 (3009.199964983091, 3405.618879856903)]
points

[(1580.2625829719882, 2006.2332587773935),
 (2587.1554413730464, 2129.3895074984475),
 (3303.883400693637, 1981.3638255553901),
 (3488.20262126068, 1972.3808912601758),
 (4413.030722076421, 2010.0458795768866),
 (2994.4072290497675, 588.0261617342247),
 (3040.4411359702813, 1434.1709473039678),
 (2893.520672163949, 2446.3018650937975),
 (3009.199964983091, 3405.618879856903)]

In [34]:
net.vertices /= mm_to_px
points = np.array(points)/mm_to_px
points, net.vertices

(array([[115.46400094, 146.58811856],
        [189.03397545, 155.58669473],
        [241.4026632 , 144.77100014],
        [254.87019377, 144.11465002],
        [322.44399692, 146.86669281],
        [218.79037246,  42.96491863],
        [222.15390148, 104.78962002],
        [211.41896113, 178.74232035],
        [219.87122351, 248.83610216]]),
 array([[109.19901575, 146.13267717,   0.        ],
        [187.84062349, 156.00688632,   0.        ],
        [241.55108125, 145.0682931 ,   0.        ],
        [255.30619848, 144.41328751,   0.        ],
        [329.19901575, 146.13267717,   0.        ],
        [219.19901575,  36.13267717,   0.        ],
        [222.62141992, 104.06494364,   0.        ],
        [211.02782111, 180.0455912 ,   0.        ],
        [219.19901575, 256.13267717,   0.        ]]))

In [35]:
file_path = os.path.join(BYU_UW_root, 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)
strain_to_stress = net.material_model(stress_data, strain_data, interpolation_kind = 'cubic')
TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive
A = [TPU_nl['A']]*len(net.edges)

vertices_equilibrium = np.copy(net.vertices)
points = np.array(points)

vertices_equilibrium[net.fixed,:2] = points[net.fixed]

vertices_equilibrium, l1_equilibrium, f = net.find_equilibrium(vertices_equilibrium, A, strain_to_stress)
# net.net_plot(color=False, elables = True, vlabels = False, custom_vertices = vertices_equilibrium)
vertices_equilibrium

array([[115.46400094, 146.58811856,   0.        ],
       [187.84062349, 156.00688632,   0.        ],
       [241.55108125, 145.0682931 ,   0.        ],
       [255.30619848, 144.41328751,   0.        ],
       [322.44399692, 146.86669281,   0.        ],
       [218.79037246,  42.96491863,   0.        ],
       [222.62141992, 104.06494364,   0.        ],
       [211.02782111, 180.0455912 ,   0.        ],
       [219.87122351, 248.83610216,   0.        ]])

In [36]:
# points = []
fig, ax = plt.subplots(figsize=(15, 10))
ax.axis('off')
ax.set_aspect('equal')
ax.imshow(aligned_image)
ax = net.net_plot_mat(ax, vlabels=True, fp=True, vertices_c=vertices_equilibrium*mm_to_px)
# ax = net_plot_mat(net, vlabels=True, fp=True, vertices_c=vertices_equilibrium)
ax.plot(net.vertices[net.fixed,0]*mm_to_px, net.vertices[net.fixed,1]*mm_to_px, 'b*', markersize=5, label='Fixed points - designed')
ax.set_xlim(np.min(net.vertices[net.fixed,0])*mm_to_px - 50, np.max(net.vertices[net.fixed,0])*mm_to_px+ 50)
ax.set_ylim(np.max(net.vertices[net.fixed,1])*mm_to_px+ 50, np.min(net.vertices[net.fixed,1])*mm_to_px- 50)
ax.legend(fontsize=12)
plt.show()
plt.savefig(os.path.join(BYU_UW_root, 'images','validation_images', model_name_im + '_val.png'), dpi=300, bbox_inches='tight')

In [37]:
vertices_real = np.array(points)
# error_pixels = np.sqrt(np.sum((vertices_real - net.vertices[:, :2])**2, axis=1))
error_pixels = np.sqrt(np.sum((vertices_real - vertices_equilibrium[:, :2])**2, axis=1))
error_mm = error_pixels / mm_to_px
error_mm

array([0.        , 0.09244119, 0.0242786 , 0.03861364, 0.        ,
       0.        , 0.06301222, 0.09942139, 0.        ])

In [38]:
error_rel = 0
for edge_i, edge in enumerate(net.edges):
    coor0, coor1 = vertices_real[edge[0]], vertices_real[edge[1]]
    l1m = np.linalg.norm(coor0[:2] - coor1[:2])
    # error_rel += np.abs(l1m - net.l1[edge_i]) / net.l1[edge_i]
    error_rel += np.abs(l1m - l1_equilibrium[edge_i]) / l1_equilibrium[edge_i]
# error_rel /= distance_between_markers_mm
error_rel/ len(net.edges) # percentage error

0.020088368262377934

In [43]:
# net.vertices /= mm_to_px # Scale the network vertices to match the image scale
net.vertices -= np.array([width/2, height/2,0])

# Save the calibration results
calibration_results = {
    'model_name': model_name,
    'model_name_im': model_name_im,
    'camera_matrix': mtx,
    'distortion_coefficients': dist,
    'homography_matrix': homography_matrix,
    'real_vertices': vertices_real,
    'equilibrium_vertices': vertices_equilibrium,
    'vertices': net.vertices,
    'error_pixels': error_pixels,
    'error_mm': error_mm,
    'error_rel': error_rel
}
np.save(os.path.join(BYU_UW_root, 'calibration_results', model_name_im + '_cr.npy'), calibration_results)

In [48]:
fig, ax = plt.subplots(1, 3, figsize=(15, 10), sharey=True)
ax[0].set_xlabel('Force (N)')
ax[1].set_xlabel('Curvature (1/m)')
ax[2].set_xlabel('Initial length (m)')
ax[0].set_ylabel('Relative error (%)')

calibration_results_files = glob.glob(os.path.join(BYU_UW_root, 'calibration_results', '*_cr.npy'))

for file in calibration_results_files:
    data = np.load(file, allow_pickle=True).item()
    model_name_im = data['model_name_im']
    model_name = data['model_name']
    real_vertices = data['real_vertices']
    vertices_equilibrium = data['equilibrium_vertices']

    net = Network_custom.load_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))

    f = net.f
    R = net.R
    kappa = 1/R
    l0 = net.l0
    error_edge = []
    for edge_i, edge in enumerate(net.edges):
        coor0, coor1 = vertices_real[edge[0]], vertices_real[edge[1]]
        l1m = np.linalg.norm(coor0[:2] - coor1[:2])
        error_edge.append(np.abs(l1m - l1_equilibrium[edge_i]) / l1_equilibrium[edge_i])
    edge_error = np.array(error_edge) * 100 # percentage error
    ax[0].plot(f, error_edge, 'o', label=model_name_im)
    ax[1].plot(kappa, error_edge, 'o', label=model_name_im)
    ax[2].plot(l0, error_edge, 'o', label=model_name_im)

ax[0].legend()
ax[0].set_title('Force vs Relative Error')
ax[1].set_title('Curvature vs Relative Error')
ax[2].set_title('Initial Length vs Relative Error')
plt.tight_layout()
plt.show()

    